In [ ]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import model_from_yaml
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys
from optparse import OptionParser
from sklearn.impute import KNNImputer
import rdkit
import rdkit.Chem as Chem
import torch
import pandas as pd
import numpy as np
import sys
import subprocess
import os
sys.path.append('jtnn/')
from jtnn import *

In [ ]:
from cmapPy.pandasGEXpress.parse_gct import parse

In [ ]:
data2 = parse('raw_data/annotated_GSE101406_Broad_LINCS_P100_Level4_DIFF_n1684x96.gctx') #for p100

In [ ]:
#data2 = parse('annotated_GSE101406_Broad_LINCS_GCP_Level4_DIFF_n1712x59.gctx') #for gcp

In [ ]:
p100 = pd.DataFrame(KNNImputer(n_neighbors=5, weights="uniform").fit_transform(data2.data_df),columns=data2.data_df.columns)
p100 = p100.T
p100.columns = data2.data_df.T.columns

p100 = p100.T

In [ ]:
testtt = pd.read_csv('raw_data/GSE101406_Broad_LINCS_pert_info.txt',sep='\t', index_col='pert_id')

In [ ]:
smiless = []
for idd in full['pert_id']:
    smiless.append(testtt.loc[idd]['canonical_smiles'])

In [ ]:
canons = []
for smiles in smiless:
    mol = Chem.MolFromSmiles(smiles)
    canonical_smiles = Chem.MolToSmiles(mol, isomericSmiles=False)
#     if canonical_smiles in canons:
#         continue;
    canons.append(canonical_smiles)

In [ ]:
canonical_smiles = pd.DataFrame(canons)
canonical_smiles.columns = ['canonical_smiles']
canonical_smiles = canonical_smiles.T
canonical_smiles.columns = data2.data_df.columns

canonical_smiles = canonical_smiles.T

In [ ]:

full = pd.concat((pd.DataFrame(data2.col_metadata_df),canonical_smiles, p100.T),axis=1)


In [ ]:
full.to_csv('raw_data/gcp_with_smiles_but_no_trash.csv')

In [ ]:
full.drop([ 'det_plate', 'det_well', 'lsm_id', 'pert_dose',
       'pert_dose_unit', 'pert_id', 'pert_iname', 'pert_time',
       'pert_time_unit'],axis=1,inplace=True)

In [ ]:
jtvae = pd.read_csv('Final Data/smiles_to_jtvae.csv')

In [ ]:
full.reset_index().merge(jtvae,on='canonical_smiles',how='left').to_csv('Final Data/GCP_no_metadata_with_jtvae.csv',index = False)

In [ ]:
canons = list(dict.fromkeys(canons))

In [ ]:
lg = rdkit.RDLogger.logger()
lg.setLevel(rdkit.RDLogger.CRITICAL)

vocab = [x.strip("\r\n ") for x in open("unique_canonical_train_vocab.txt")]
vocab = Vocab(vocab)

hidden_size = 450
latent_size = 56
depth = 3
stereo = True

model = JTNNVAE(vocab, hidden_size, latent_size, depth, stereo=stereo)
model.load_state_dict(torch.load("Models/model.iter-9-6000"))
# opts.model_path #MPNVAE-h450-L56-d3-beta0.001/model.iter-4

model = model.cuda()

In [ ]:
def jtvae_representation(smiles, mol):
    koku = pd.DataFrame(columns=list(range(56)))

#     smiles3D = Chem.MolToSmiles(mol, isomericSmiles=False)
    dec_smiles = model.reconstruct(smiles, DataFrame=koku)

    
    del dec_smiles
    torch.cuda.empty_cache()

    return koku.values

In [ ]:
jtvaes = []
for smile in canons:
    jtvaes.append(jtvae_representation(smiles=smile,mol=None))
    torch.cuda.empty_cache()